In [1]:
import io
import numpy as np
import PIL
import requests
import openpifpaf
import torch
import cv2
import json
import time
from PIL import Image
from pymongo import MongoClient
from datetime import datetime

print('OpenPifPaf version', openpifpaf.__version__)
print('PyTorch version', torch.__version__)

OpenPifPaf version 0.13.4
PyTorch version 1.11.0


In [2]:
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
#predictions, gt_anns, image_meta = predictor.pil_image(pil_im)

In [3]:
url = "http://59.152.163.68:5000/stream?src=0"
cap = cv2.VideoCapture(url)
prev = 0
FPS = 10

In [4]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())

['admin', 'config', 'local', 'mydb']


In [5]:
def calcul_xy(xy):
    result = []
    
    neck = [((xy[5][0]+xy[6][0])/2 + xy[0][0]) / 2, ((xy[5][1]+xy[6][1])/2 + xy[0][1]) / 2]
    hip = [(xy[11][0]+xy[12][0])/2, (xy[11][1]+xy[12][1])/2]
    
    result.append((xy[0][0] - neck[0]).item())
    result.append((xy[0][1] - neck[1]).item())
    
    result.append((neck[0] - xy[7][0]).item())
    result.append((neck[1] - xy[7][1]).item())
    
    result.append((neck[0] - xy[8][0]).item())
    result.append((neck[1] - xy[8][1]).item())
    
    result.append((xy[7][0] - xy[9][0]).item())
    result.append((xy[7][1] - xy[9][1]).item())
    
    result.append((xy[8][0] - xy[10][0]).item())
    result.append((xy[8][1] - xy[10][1]).item())
    
    # v5 neck - hip
    result.append((neck[0] - hip[0]).item())
    result.append((neck[1] - hip[1]).item())
    
    result.append((hip[0] - xy[13][0]).item())
    result.append((hip[1] - xy[13][1]).item())
    
    result.append((hip[0] - xy[14][0]).item())
    result.append((hip[1] - xy[14][1]).item())
    
    result.append((xy[13][0] - xy[15][0]).item())
    result.append((xy[13][1] - xy[15][1]).item())
    
    result.append((xy[14][0] - xy[16][0]).item())
    result.append((xy[14][1] - xy[16][1]).item())
    
    return result    

In [6]:

db = client['mydb']
db = db.xy

def input_db(db, data):
    input_data = dict()
    
    input_data['date'] = datetime.now()
    
    for i in range(10):
        
        for j in range(2):
            index = "v" + str(i)
            if j%2 == 0:
                index += 'x'
            else:
                index += 'y'
            
            input_data[index] = data[i*2 + j]
    
    #print(type(input_data['v0x']))
    db.insert_one(input_data)
            

In [7]:
if cap.isOpened(): 
    ret, img = cap.read()
    cur = time.time() - prev
    before_xy = None
    before_chk = False
    
    cnt = 0
    
    while ret:
        try:
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #print(type(img))
            img = Image.fromarray(img)

            predictions, gt_anns, image_meta = predictor.pil_image(img)

            if len(predictions) < 1:
                ret, img = cap.read()
                cur = time.time() - prev
                continue

            xy = predictions[0].data
            
            
            ## 좌표 중 0으로 출력되는 포인트를 인식하지 못한 부분은 이전 값으로 채우기
            if before_chk:
                for i in range(len(xy)):
                    if xy[i][0] == 0:
                        xy[i][0], xy[i][1] = before_xy[i][0], before_xy[i][1]
            
            before_xy = xy
            before_chk = True
            
            result = calcul_xy(xy)
            input_db(db, result)

            ret, img = cap.read()
            cur = time.time() - prev
            cnt+=1

            if cnt%100 == 0:
                print(cnt)
                
        except:
            print("Frame exception")
            cap.release()
            cap = cv2.VideoCapture(url)
else:
    print("url is not opened!!")

src/openpifpaf/csrc/src/cif_hr.cpp:102: UserInfo: resizing cifhr buffer
src/openpifpaf/csrc/src/occupancy.cpp:53: UserInfo: resizing occupancy buffer
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
